In [ ]:
import glob
import imageio.v3 as iio
import ipympl
import matplotlib.pyplot as plt
import numpy as np
import skimage as ski

%matplotlib widget

## Simple thresholding

1. 이미지를 gray scale 로 읽는다.
2. Blurring을 이용해서 노이즈를 제거한다. (de-noize)
3. 노이즈가 제거된 이미지의 histogram을 분석한다.
4. threshold 값을 찾고 이를 이용하여 필요없는 부분을 masking 처리한다.
5. 4에서 생성된 mask를 이용해 원본 이미지에서 원하는 부분만 추출한다.

In [ ]:
shape01 = iio.imread(uri="data/shapes-01.jpg")

fig, ax = plt.subplots()
ax.imshow(shape01)

In [ ]:
gray_shapes = ski.color.rgb2gray(shape01)
blurred_shapes = ski.filters.gaussian(gray_shapes, sigma=1)

fig, ax = plt.subplots()
ax.imshow(blurred_shapes, cmap='gray')

In [ ]:
histogram, bin_edges = np.histogram(blurred_shapes, bins=256, range=(0.0,1.0))

fig, ax = plt.subplots()
ax.plot(bin_edges[0:-1], histogram)

In [ ]:
t = 0.8
binary_mask = blurred_shapes < t

fig, ax = plt.subplots()
ax.imshow(binary_mask, cmap="gray")

In [ ]:
selection = shape01.copy()
selection[~binary_mask] = 0

fig, ax = plt.subplots()
ax.imshow(selection)

## Practice

In [ ]:
# 0. 이미지 읽기
import numpy as np
shapes02 = iio.imread(uri="data/shapes-02.jpg")
# 1. Gray 이미지로 변경
gray_shapes02= ski.color.rgb2gray(shapes02)
# 2. Histogram 계산
histogram, bin_edges = np.histogram(gray_shapes02, bins=256, range=(0., 1.))
fig, ax = plt.subplots()
ax.plot(bin_edges[0:-1], histogram)

In [ ]:
# 3. 임계값 설정
t2 = 0.6
binary_mask2 = gray_shapes02 > t2
# 4. 이미지 출력
fig, ax = plt.subplots()
ax.imshow(binary_mask2, cmap='gray')

In [ ]:
selection2 = shapes02.copy()
selection2[~binary_mask2] = 0

fig, ax = plt.subplots()
ax.imshow(selection2)

## Automatic thresholding

매번 수동으로 임계값을 구하는 것은 실용적이지 못하다. 이를 위해 알고리즘들이 개발되었으며, 이 장에서는 간단하 otsu's method를 살펴본다.

In [ ]:
maize_roots = iio.imread(uri="data/maize-root-cluster.jpg")

fig, ax = plt.subplots()
ax.imshow(maize_roots)

In [ ]:
gray_maize_roots = ski.color.rgb2gray(maize_roots)
blurred_maize_roots = ski.filters.gaussian(gray_maize_roots, sigma=1)
histogram, bin_edges = np.histogram(blurred_maize_roots, bins=256, range=(0.0, 1.0))

fig, ax = plt.subplots()
ax.plot(bin_edges[0:-1], histogram)

In [ ]:
t = ski.filters.threshold_otsu(blurred_maize_roots)
binary_mask3 = blurred_maize_roots > t
print(t)
fig, ax = plt.subplots()
ax.imshow(binary_mask3, cmap='gray')

In [ ]:
import matplotlib.pyplot as plt
selection3 = maize_roots.copy()
selection3[~binary_mask3] = 0

fig, ax = plt.subplots()
ax.imshow(selection3)

In [ ]:
# measure root mass
def measure_root_mass(filename, sigma=1.0):
    image = iio.imread(filename, mode="L")
    blurred_image = ski.filters.gaussian(image, sigma=sigma)
    t = ski.filters.threshold_otsu(blurred_image)
    binary_mask = blurred_image > t
    h = image.shape[0]
    w = image.shape[1]
    r = np.count_nonzero(binary_mask)
    return r / (h * w)

In [ ]:
all_files = glob.glob("data/trial-*.jpg")
for filename in all_files:
    density = measure_root_mass(filename=filename, sigma=1.5)
    print(filename, density, sep=",")

In [ ]:
## enhance root mass

import glob
def enhanced_root_mass(filename, sigma=1.0):
    image = iio.imread(uri=filename, mode="L")
    blurred_image = ski.filters.gaussian(image, sigma=sigma)
    binary_mask = blurred_image < 0.95
    t = ski.filters.threshold_otsu(blurred_image[binary_mask])
    binary_mask = (blurred_image < 0.95) & (blurred_image > t)
    pixels = np.count_nonzero(binary_mask)
    w = image.shape[1]
    h = image.shape[0]
    return pixels / (w * h)

all_files = glob.glob("data/trial-*.jpg")
for f in all_files:
    density = enhanced_root_mass(filename=f, sigma=1.0)
    print(f, density, sep=",")

## Practice 2. Thresholding a bacteria colony image

In [ ]:
petri_dish = iio.imread(uri="data/colonies-01.tif")
gray_dish = ski.color.rgb2gray(petri_dish)
blurred_dish = ski.filters.gaussian(gray_dish, sigma=1)
histogram, bin_edges = np.histogram(blurred_dish, bins=256, range=(0., 1.))

fig, ax = plt.subplots()
ax.plot(bin_edges[0:-1], histogram)

In [ ]:
import matplotlib.pyplot as plt
binary_mask = blurred_dish < 0.3
fig, ax = plt.subplots()
ax.imshow(binary_mask, cmap='gray')